In [45]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (170 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

In [46]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [47]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [48]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.createOrReplaceTempView('home_sales')


In [49]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = ("""
SELECT YEAR(date) AS year,
       ROUND(AVG(price), 2) AS Avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
""")

spark.sql(query).show()


+----+---------+
|year|Avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [50]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = ("""
SELECT YEAR(date_built) AS year_built,
       ROUND(AVG(price), 2) AS Avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
""")


spark.sql(query).show()

+----------+---------+
|year_built|Avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [51]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = ("""
SELECT YEAR(date_built) AS year_built,
       ROUND(AVG(price), 2) AS Avg_price
FROM home_sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built
""")

spark.sql(query).show()


+----------+---------+
|year_built|Avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [63]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = ("""
SELECT view,
       ROUND(AVG(price), 2) AS Avg_Price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC
""")

result = spark.sql(query)

# Show the results
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 401393.34|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.4513521194458008 seconds ---


In [53]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [54]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [64]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = ("""
SELECT view, ROUND(AVG(price), 2) AS Avg_Price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC
""")

# Show the results
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 401393.34|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.4314882755279541 seconds ---


In [56]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet('p_home_sales', mode='overwrite')

In [57]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('p_home_sales')

In [58]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('partitioned_home_sales')

In [59]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = ("""
SELECT ROUND(AVG(price), 2) AS Avg_Price, view
FROM partitioned_home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY Avg_Price DESC
""")

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| Avg_Price|view|
+----------+----+
|1137372.73|  91|
|1129040.15|  97|
|1117233.13|  84|
|1114042.94|  75|
|1107839.15|  89|
|1080649.37|  78|
|1076205.56|  77|
| 1072285.2|  87|
|1070444.25|  86|
| 1063498.0|  82|
|1062654.16|  90|
|1061201.42|  99|
|1058802.78|  76|
|1056336.74|  85|
| 1054325.6|  95|
|1053739.33|  98|
|1053472.79|  81|
|1033965.93|  83|
| 1033536.2|  94|
|1031719.35|  88|
+----------+----+
only showing top 20 rows

--- 0.5376317501068115 seconds ---


In [60]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [61]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
